# Random Forest (RF) with Genetic Algorithm (GA)

In most of machine learning tasks, we will not know the true objective function but rather we tend to find the hypothesis function that generalise well our dataset. Therfore, in this notebook, we will look at how to use Genetic Algorithm to optimise parameters for Random Forest. 

The dataset that we will use for this task is the collection of data from users that interact with mobile ads. The objective for this assignment is to find the probability of mobile users cliking the ads which is used for bid optimosation. Bid optimosation is a very big challenge for mobile platform because we want to show ads to mobiles users at the right time and right content. So, they would eventually participate in buying products or useing services. As a result, we need to find who we would push the ads to at what time and who are our potential customers for our product based upon the context that they are in routinely in order to improve the conversion rate, click through rate etc.

### import library

First of all, let us import all the library that we will use later on.

In [24]:
import pandas as pd
import sklearn
from sklearn import cross_validation
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")
import ast

### read "train data" in

Now we are ready to start. We Then load training data and label for training data into our work space.

In [25]:
X_train = pd.read_csv("train.csv",dtype={'id':str, 'ad_format': str, 'ad_format_id': str, 'advertiser_id': str, 'age_rating': str, 'app_age_rating': str, 'app_category_id': str, 'app_company': str, 'app_genre_id': str, 'app_id': str, 'app_quality': str, 'brand_vertical_id': str, 'campaign_id': str, 'country_id': str, 'creative_id': str, 'creative_type': str, 'make': str, 'device_model': str, 'device_os_ver': str, 'device_wh': str, 'device_ww': str, 'genre_id': str, 'hour_local': str, 'isp_type': str, 'language': str, 'line_item_id': str, 'orientation': str, 'platform_id': str, 'publisher': str, 'sdepth': str, 'viewer_token': str, 'ed': str, 'edc': str, 'campaign_type': str, 'city': str, 'isp_name': str, 'ssp': str})
y_train = pd.read_csv("train.labels.csv")

We then inspect what columns our dataset has including the label.

In [26]:
#list column name for training and test set
print(X_train.columns.values)
print(y_train.columns.values)

['id' 'ad_format' 'ad_format_id' 'advertiser_id' 'age_rating'
 'app_age_rating' 'app_category_id' 'app_company' 'app_genre_id' 'app_id'
 'app_quality' 'brand_vertical_id' 'campaign_id' 'country_id' 'creative_id'
 'creative_type' 'make' 'device_model' 'device_os_ver' 'device_wh'
 'device_ww' 'genre_id' 'hour_local' 'isp_type' 'language' 'line_item_id'
 'orientation' 'platform_id' 'publisher' 'sdepth' 'viewer_token' 'ed' 'edc'
 'campaign_type' 'city' 'isp_name' 'ssp']
['id' 'label']


### read "test" data

In [28]:
test = pd.read_csv("test.csv",dtype={'id': str, 'ad_format': str, 'ad_format_id': str, 'advertiser_id': str, 'age_rating': str, 'app_age_rating': str, 'app_category_id': str, 'app_company': str, 'app_genre_id': str, 'app_id': str, 'app_quality': str, 'brand_vertical_id': str, 'campaign_id': str, 'country_id': str, 'creative_id': str, 'creative_type': str, 'make': str, 'device_model': str, 'device_os_ver': str, 'device_wh': str, 'device_ww': str, 'genre_id': str, 'hour_local': str, 'isp_type': str, 'language': str, 'line_item_id': str, 'orientation': str, 'platform_id': str, 'publisher': str, 'sdepth': str, 'viewer_token': str, 'ed': str, 'edc': str, 'campaign_type': str, 'city': str, 'isp_name': str, 'ssp': str})

## add logical "train" column to train dataset as well as test dataset

We add train dataset to training set and test set. This is because later we will merge them together and encode data from string to number in order for RF to calculate the information gain of the classifier. This is to ensure consistancy of data encoding when we make peodiction from classifier that we build.

In [27]:
X_train["train"] = True
test["train"] = False
idx =  test["id"]

### merging "train" and "test"

Now that everything is ready we will merge training and test set.

In [30]:
print(X_train.shape)
print(test.shape)
print(X_train.columns.values)
print(test.columns.values)

(201103, 38)
(201347, 38)
['id' 'ad_format' 'ad_format_id' 'advertiser_id' 'age_rating'
 'app_age_rating' 'app_category_id' 'app_company' 'app_genre_id' 'app_id'
 'app_quality' 'brand_vertical_id' 'campaign_id' 'country_id' 'creative_id'
 'creative_type' 'make' 'device_model' 'device_os_ver' 'device_wh'
 'device_ww' 'genre_id' 'hour_local' 'isp_type' 'language' 'line_item_id'
 'orientation' 'platform_id' 'publisher' 'sdepth' 'viewer_token' 'ed' 'edc'
 'campaign_type' 'city' 'isp_name' 'ssp' 'train']
['id' 'ad_format' 'ad_format_id' 'advertiser_id' 'age_rating'
 'app_age_rating' 'app_category_id' 'app_company' 'app_genre_id' 'app_id'
 'app_quality' 'brand_vertical_id' 'campaign_id' 'country_id' 'creative_id'
 'creative_type' 'make' 'device_model' 'device_os_ver' 'device_wh'
 'device_ww' 'genre_id' 'hour_local' 'isp_type' 'language' 'line_item_id'
 'orientation' 'platform_id' 'publisher' 'sdepth' 'viewer_token' 'ed' 'edc'
 'campaign_type' 'city' 'isp_name' 'ssp' 'train']


In [31]:
#concat X_train and X_test
frames = [X_train, test]
data = pd.concat(frames)

In [32]:
#check no. before and after concat
print(X_train.shape[0]+test.shape[0])
print(data.shape[0])

402450
402450


### delete empty column ["app_quality","device_wh","device_ww","ed","edc"]

During the data visualisation process, we found that there are some columns that are all empty or has only one value. We therefore will remove them because they will not provide information to split up nodes in the trees. This will also reduce the workload that the algorithm will have to do too.

In [33]:
data.drop(["app_quality","device_wh","device_ww","ed","edc"],inplace=True,axis=1)
print(data.columns.values)
print(data.shape)

['id' 'ad_format' 'ad_format_id' 'advertiser_id' 'age_rating'
 'app_age_rating' 'app_category_id' 'app_company' 'app_genre_id' 'app_id'
 'brand_vertical_id' 'campaign_id' 'country_id' 'creative_id'
 'creative_type' 'make' 'device_model' 'device_os_ver' 'genre_id'
 'hour_local' 'isp_type' 'language' 'line_item_id' 'orientation'
 'platform_id' 'publisher' 'sdepth' 'viewer_token' 'campaign_type' 'city'
 'isp_name' 'ssp' 'train']
(402450, 33)


### encode categorical data to number

Now we are ready to encode data.

In [34]:
dataEncoded = data.apply(preprocessing.LabelEncoder().fit_transform)
dataEncoded.head()

,id,ad_format,ad_format_id,advertiser_id,age_rating,app_age_rating,app_category_id,app_company,app_genre_id,app_id,...,orientation,platform_id,publisher,sdepth,viewer_token,campaign_type,city,isp_name,ssp,train
0,0,3,440,5,3,0,20,51,0,694,...,1,1,51,2,147291,2,4024,359,0,1
1,111111,3,440,5,3,0,20,51,0,694,...,1,1,51,916,59410,2,7533,2398,0,1
2,222222,3,440,5,3,0,20,51,0,694,...,1,1,51,916,50831,2,1197,363,0,1
3,333333,3,446,5,3,0,19,41,0,8,...,3,1,41,916,248296,2,10466,2398,0,1
4,346895,3,446,5,3,0,19,41,0,8,...,3,1,41,2066,67715,2,1003,2974,0,1


### split data back to "training set" and "test set"

After we decoded our dataset, we now will split data back into training set and test set.

In [35]:
X_train = dataEncoded[dataEncoded["train"]==1]
test =  dataEncoded[dataEncoded["train"]==0]

### delete id column in train

Now we will delete "id" column from train set which occur during the merging step. 

In [36]:
#remove 'íd' field
X_train = X_train.iloc[:,1:]
y_train = y_train.iloc[:,1:]
#list column name for training and test set
print(X_train.columns.values)
print(y_train.columns.values)

['ad_format' 'ad_format_id' 'advertiser_id' 'age_rating' 'app_age_rating'
 'app_category_id' 'app_company' 'app_genre_id' 'app_id'
 'brand_vertical_id' 'campaign_id' 'country_id' 'creative_id'
 'creative_type' 'make' 'device_model' 'device_os_ver' 'genre_id'
 'hour_local' 'isp_type' 'language' 'line_item_id' 'orientation'
 'platform_id' 'publisher' 'sdepth' 'viewer_token' 'campaign_type' 'city'
 'isp_name' 'ssp' 'train']
['label']


### further split "training data" to "train", "validation" and "test" set

For our current traing set, we would spit it into 3 more chunk
* the first chunk will be our actual "training set"
* the second chunk will be used to evaluate the perfromnace of our classifier whether it performs better than our existing population.
* the third chunk will be used for final evaluation of the parameter after the algorithm finishes running.

In [37]:
 #set seed
#random.seed(9001)
#split data to train and test
#X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_train, y_train, test_size=0.25, random_state=0)
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_train, y_train, test_size=0.25)
def getTrainData():
    #split train one more time to train and validation
    #X_train, X_valid, y_train, y_valid 
    return cross_validation.train_test_split(X_train, y_train, test_size=0.25)

In [38]:
#check size of training, validation and test dataset
print([X_train.shape, y_train.shape])
print([X_test.shape, y_test.shape])
#print([X_valid.shape, y_valid.shape])

[(150827, 32), (150827, 1)]
[(50276, 32), (50276, 1)]


### set parameters' values

After everything is set, now let's create a function to make the DNA string. In other words, this function will generate parameter for RF at random to evaluate.

In [50]:
def generateParam(trees):
    #The number of trees in the forest.
    n_estimators = random.randint(50, trees)
    #The function to measure the quality of a split.
    criterion = random.choice(["gini", "entropy"])
    #The number of features to consider when looking for the best split:
    max_features = random.choice(["auto", "sqrt", "log2", None])
    #The maximum depth of the tree
    max_depth = None if random.uniform(0, 1) < 0.3 else random.randint(5, 300)
    #The minimum number of samples required to split an internal node.
    min_samples_split = random.randint(10, 100)
    #The minimum number of samples in newly created leaves.
    min_samples_leaf = random.randint(10, 50)
    #The minimum weighted fraction of the input samples required to be at a leaf node.
    min_weight_fraction_leaf = random.uniform(0, 0.5)
    #Grow trees with max_leaf_nodes in best-first fashion.
    max_leaf_nodes = None if random.uniform(0, 1) < 0.1 else random.randint(2, 1000)
    #Whether bootstrap samples are used when building trees.
    bootstrap = True
    #Whether to use out-of-bag samples to estimate the generalization error.
    oob_score = random.choice([True, False])
    #The number of jobs to run in parallel for both fit and predict.
    n_jobs = random.choice([1,2])
    #the state of the random number generator
    #random_state = random.choice(["RandomState", None])
    #Controls the verbosity of the tree building process.
    #verbose =
    #When set to True, reuse the solution of the previous call to fit and add more estimators to the ensemble.
    #otherwise, just fit a whole new forest.
    #warm_start = random.choice([True, False])
    #Weights associated with classes
    #class_weight = random.choice(["balanced", "balanced_subsample",None])
    return [n_estimators, criterion, max_features, max_depth, min_samples_split, min_samples_leaf, min_weight_fraction_leaf, max_leaf_nodes, bootstrap, oob_score, n_jobs]  

In [51]:
#print generateParam()

### evalLoss

This is the "LogLoss" function that use to eveluate how well our classifier is doing. 

In [52]:
def EvalLogLoss(predicted, actual):
    return sklearn.metrics.log_loss(actual, predicted, eps=1e-15)

### get first generation

This function is used to create our first population.

In [53]:
def getFistGen(n, trees):
    pop = list()
    #n = 2 #50
    for i in range(0,n):
        pop.extend([generateParam(trees)])
    return pop

### computeProb

This function is used to eveluate parameter that we randomly generate from GA process.

In [54]:
def computeProb(dna,X_train,y_train,X_valid):
    clf =  RandomForestClassifier(n_estimators=dna[0], criterion=dna[1], max_features=dna[2], \
                              max_depth=dna[3], min_samples_split=dna[4], min_samples_leaf=dna[5], \
                              min_weight_fraction_leaf=dna[6], max_leaf_nodes=dna[7], bootstrap=dna[8], \
                              oob_score=dna[9], n_jobs=dna[10])
    #clf =  RandomForestClassifier(n_estimators=dna[0])
    clf.fit(X_train.as_matrix(), y_train.as_matrix())
    predicted = pd.DataFrame(clf.predict_proba(X_valid))
    return predicted[0].as_matrix()

### evalFirstGen

This function is used for eveluate our population in first generation. 

In [55]:
def computeFirstGen(pop):
    loss_lst = list()
    for dna in pop:
        X_train, X_valid, y_train, y_valid = getTrainData()
        predicted = computeProb(dna, X_train, y_train, X_valid)
        #loss = EvalLogLoss(predicted[0].as_matrix(), y_valid)
        loss = EvalLogLoss(predicted, y_valid)
        loss_lst.extend([loss])
    return loss_lst

### *run RF*

This function perform GA search through our parameter space and try to find to optimal one and write into csv file.

In [ ]:
def runRF(numberOfPop, numberOfTrees, numberOfGeneration, fileNames):
    print "start RF"
    for f in fileNames:
        print "start "+f
        pop = getFistGen(numberOfPop, numberOfTrees)
        popLoss = computeFirstGen(pop)
        for i in range(0,numberOfGeneration):
            #create training dataset and validation dataset
            X_train, X_valid, y_train, y_valid = getTrainData()
            #hybridge and produce offspring
            parent1 = random.choice(pop)
            parent2 = random.choice(pop)
            pos = random.randint(1, len(parent1)-1)
            offspring1 = parent1[0:pos]+parent2[pos:]
            offspring2 = parent2[0:pos]+parent1[pos:]
            #mutate offspring 
            newParam = generateParam(numberOfTrees)
            pos = random.randint(0, len(pop[0])-1)
            offspring1[pos] = newParam[pos]
            pos = random.randint(0, len(pop[0])-1)
            offspring2[pos] = newParam[pos]
            #evaluate offsprings
            offspring1Predicted = computeProb(offspring1,X_train,y_train,X_valid)
            offspring1Loss = EvalLogLoss(offspring1Predicted, y_valid)
            offspring2Predicted = computeProb(offspring2,X_train,y_train,X_valid)
            offspring2Loss = EvalLogLoss(offspring2Predicted, y_valid)
            #print(offspring2Loss)
            #add offsprings to pop
            pop.extend([offspring1,offspring2])
            popLoss.extend([offspring1Loss,offspring2Loss])
            mapPopToLoss = zip(popLoss,pop)
            sortedMapPopToLoss = sorted(mapPopToLoss)
            newPopLoss = [key[0] for key in sortedMapPopToLoss]
            newPop = [value[1] for value in sortedMapPopToLoss]
            pop = pop[0:numberOfPop]
            popLoss = popLoss[0:numberOfPop]
            if i%25==0:
                print i
                print(len(sortedMapPopToLoss))
        df = pd.DataFrame(sorted(zip(popLoss,pop)))
        df.to_csv(f)
    return "finish RF"

#runRF(numberOfPop, numberOfTrees, numberOfGeneration, fileNames)
print runRF(50, 5000, 100, ["third.csv","fourth.csv","fifth.csv","sixth.csv","seventh.csv","eight.csv"])


start RF
start third.csv


### test params on test set

After we sucessfully run GA+RF, let's inspect our results. We will use the DNA string that provides the best logLoss on our third chunk ("test set") that we left out previously. 

In [57]:
firstrun = pd.read_csv("first_run.csv")
sortfrun = firstrun.sort(columns=["0"])
sortfrun.head()

,Unnamed: 0,0,1
28,28,1.770681,"[786, 'entropy', 'sqrt', 61, 109, 487, 0.48675..."
40,40,1.800684,"[373, 'gini', 'log2', 25, 144, 355, 0.48044017..."
23,23,1.806080,"[975, 'entropy', 'sqrt', None, 76, 965, 0.4468..."
7,7,1.809603,"[272, 'gini', 'auto', 5, 156, 619, 0.443056808..."
10,10,1.833851,"[148, 'entropy', 'sqrt', 224, 208, 705, 0.4058..."


In [ ]:
def computeLossOnTest(dna,X_train, y_train, X_test, y_test):
    clf =  RandomForestClassifier(n_estimators=dna[0], criterion=dna[1], max_features=dna[2], \
                              max_depth=dna[3], min_samples_split=dna[4], min_samples_leaf=dna[5], \
                              min_weight_fraction_leaf=dna[6], max_leaf_nodes=dna[7], bootstrap=dna[8], \
                              oob_score=dna[9], n_jobs=dna[10])
    #clf =  RandomForestClassifier(n_estimators=dna[0])
    clf.fit(X_train.as_matrix(), y_train.as_matrix())
    predicted = pd.DataFrame(clf.predict_proba(X_test))
    predicted = predicted[0].as_matrix()
    return EvalLogLoss(predicted, y_test)

In [ ]:
dna = ast.literal_eval(sortfrun["1"][28])
print computeLossOnTest(dna,X_train, y_train, X_test, y_test)

### make submission

This function will generate a submission file that use the best DNA found so far to make the prediction.

In [52]:
def makeSubmissionFile(dna,X_train, y_train, test,idx):
    test = test.iloc[:,1:]
    #test.set_index(keys= ["id"], drop=True, inplace=True)
    clf =  RandomForestClassifier(n_estimators=dna[0], criterion=dna[1], max_features=dna[2], \
                              max_depth=dna[3], min_samples_split=dna[4], min_samples_leaf=dna[5], \
                              min_weight_fraction_leaf=dna[6], max_leaf_nodes=dna[7], bootstrap=dna[8], \
                              oob_score=dna[9], n_jobs=dna[10])
    #clf =  RandomForestClassifier(n_estimators=dna[0])
    clf.fit(X_train.as_matrix(), y_train.as_matrix())
    submission = pd.DataFrame(clf.predict_proba(test), index=idx, columns=["prediction","1_pred"])
    submission = submission["prediction"]
    submission.to_csv("submission1.csv",header=True)
    return "write sucessfully"
print makeSubmissionFile(dna,X_train, y_train, test, idx)

write sucessfully


## side note and recommodation

If you run this on a single machine, the algorithm can be finished on a small number of trees e.g.500 in roughtly one night. Further, with the dataset, I tried I found that there is not much improvement for one GA. Therefore, I recommend that if you want to go with small number of trees, try multiple interations. However, althogh I did not try, you can take an advantage of parallel compotation (say Condors) to distribute your jobs in to cluster. This could help when you want to use a big number of trees.

*last edit 24/10/2016*